## 7.30 模板

A template is similar to a macro in the sense that the template parameters are replaced by
their values before compilation. The following example illustrates the difference between a
function parameter and a template parameter:

```cpp
// Example 7.46
int Multiply (int x, int m) {
   return x * m;
}

template <int m>
int MultiplyBy (int x) {
   return x * m;
}

int a, b;
a = Multiply(10,8);
b = MultiplyBy<8>(10);
```

a and b will both get the value 10 * 8 = 80. The difference lies in the way m is transferred to
the function. In the simple function, m is transferred at runtime from the caller to the called
function. But in the template function, m is replaced by its value at compile time so that the
compiler sees the constant 8 rather than the variable m. The advantage of using a template
parameter rather than a function parameter is that the overhead of parameter transfer is
avoided. The disadvantage is that the compiler needs to make a new instance of the
template function for each different value of the template parameter. If MultiplyBy in this
example is called with many different factors as template parameters then the code can
become very big.

In the above example, the template function is faster than the simple function because the
compiler knows that it can multiply by a power of 2 by using a shift operation. x*8 is
replaced by x<<3, which is faster. In the case of the simple function, the compiler doesn't
know the value of m and therefore cannot do the optimization unless the function can be 
inlined. (In the above example, the compiler is able to inline and optimize both functions and
simply put 80 into a and b. But in more complex cases it might not be able to do so).

A template parameter can also be a type. The example on page 38 shows how you can
make arrays of different types with the same template.

Templates are efficient because the template parameters are always resolved at compile
time. Templates make the source code more complex, but not the compiled code. In
general, there is no cost in terms of execution speed to using templates.

Two or more template instances will be joined into one if the template parameters are
exactly the same. If the template parameters differ then you will get one instance for each
set of template parameters. A template with many instances makes the compiled code big
and uses more cache space.

Excessive use of templates makes the code difficult to read. If a template has only one
instance then you may as well use a #define, const or typedef instead of a template
parameter.

Templates may be used for metaprogramming, as explained at page 154.

### Using templates for polymorphism

A template class can be used for implementing a compile-time polymorphism, which is more
efficient than the runtime polymorphism that is obtained with virtual member functions. The
following example shows first the runtime polymorphism:

```cpp
// Example 7.47a. Runtime polymorphism with virtual functions
class CHello {
public:
   void NotPolymorphic(); // Non-polymorphic functions go here
   virtual void Disp(); // Virtual function
   void Hello() {
     cout << "Hello ";
     Disp(); // Call to virtual function
   }
};

class C1 : public CHello {
public:
   virtual void Disp() {
     cout << 1;
   }
};

class C2 : public CHello {
public:
   virtual void Disp() {
     cout << 2;
   }
};

void test () {
   C1 Object1; C2 Object2;
   CHello * p;
   p = &Object1;
   p->NotPolymorphic(); // Called directly
   p->Hello(); // Writes "Hello 1"
   p = &Object2;
   p->Hello(); // Writes "Hello 2"
}
```

The dispatching to C1::Disp() or C2::Disp() is done at runtime here if the compiler
doesn't know what class of object p points to (see page 75). Current compilers are not very
good at optimizing away p and inlining the call to Object1.Hello(), though future
compilers may be able to do so.

If it is known at compile-time whether the object belongs to class C1 or C2, then we can
avoid the inefficient virtual function dispatch process. This can be done with a special trick
which is used in the Active Template Library (ATL) and Windows Template Library (WTL):

```cpp
// Example 7.47b. Compile-time polymorphism with templates
// Place non-polymorphic functions in the grandparent class:
class CGrandParent {
public:
   void NotPolymorphic();
};

// Any function that needs to call a polymorphic function goes in the
// parent class. The child class is given as a template parameter:
template <typename MyChild>
class CParent : public CGrandParent {
public:
   void Hello() {
     cout << "Hello ";
     // call polymorphic child function:
     (static_cast<MyChild*>(this))->Disp();
   }
};

// The child classes implement the functions that have multiple
// versions:
class CChild1 : public CParent<CChild1> {
public:
   void Disp() {
     cout << 1;
   }
};

class CChild2 : public CParent<CChild2> {
public:
   void Disp() {
     cout << 2;
   }
};

void test () {
   CChild1 Object1; CChild2 Object2;
   CChild1 * p1;
   p1 = &Object1;
   p1->Hello(); // Writes "Hello 1"
   CChild2 * p2;
   p2 = &Object2;
   p2->Hello(); // Writes "Hello 2"
}
```


Here CParent is a template class which gets information about its child class through a
template parameter. It can call the polymorphic member of its child class by type-casting its
'this' pointer to a pointer to its child class. This is only safe if it has the correct child class
name as template parameter. In other words, you must make sure that the declaration
```cpp
class CChild1 : public CParent<CChild1> {
```    
has the same name for the child class name and the template parameter.

The order of inheritance is now as follows. The first generation class (CGrandParent)
contains any non-polymorphic member functions. The second generation class
(CParent<>) contains any member functions that need to call a polymorphic function. The
third generations classes contain the different versions of the polymorphic functions. The
second generation class gets information about the third generation class through a
template parameter.

No time is wasted on runtime dispatch to virtual member functions if the class of the object
is known. This information is contained in p1 and p2 having different types. A disadvantage
is that CParent::Hello() has multiple instances that take up cache space.

The syntax in example 7.47b is admittedly very kludgy. The few clock cycles that we may
save by avoiding the virtual function dispatch mechanism is rarely enough to justify such a
complicated code that is difficult to understand and therefore difficult to maintain. If the
compiler is able to do the devirtualization (see page 75) automatically then it is certainly
more convenient to rely on compiler optimization than to use this complicated template
method.
